In [ ]:
# export
from nbdev.imports import *
from nbdev.sync import *
from nbdev.export import *
from nbdev.export import _mk_flag_re

from nbconvert.preprocessors import ExecutePreprocessor

In [ ]:
# default_exp test

# Extract tests

> The functions that grab the cells containing tests (filtering with potential flags) and execute them

Everything that is not an exported cell is considered a test, so you should make sure your notebooks can all run smoothly (and fast) if you want to use this functionality as the CLI. You can mark some cells with special flags (like slow) to make sure they are only executed when you authorize it. Those flags should be configured in your `settings.ini` (separated by a `|` if you have several of them). You can also apply a flag to one entire notebook by putting `# all_flag` in one of its cells.

## Detect flags

The following functions detect the cells that should be excluded from the tests (unless their special flag is passed).

In [ ]:
#export
class _ReTstFlags():
    "Provides test flag matching regular expressions"
    def __init__(self, all_flag): self.all_flag = all_flag
    
    @property
    def re(self):
        "Compile at first use but not before since patterns need `Config().tst_flags`"
        if not hasattr(self, '_re'):
            tst_flags = Config().get('tst_flags', '')
            _re_all, _re_magic_all = ('all_', '[ \t]+all') if self.all_flag else ('', '')
            self._re = _mk_flag_re(False, f"{_re_all}({tst_flags})", 0,
                "Matches any line with a test flag and catches it in a group")
            self._re_magic = _mk_flag_re(True, f"({tst_flags})_test{_re_magic_all}", 0,
                "Matches any line with a magic test flag and catches it in a group")
        return self._re, self._re_magic

In [ ]:
#export
_re_all_flag = _ReTstFlags(True)

In [ ]:
# export
def check_all_flag(cells):
    "Check for an `# all_flag` cell and then return said flag"
    for cell in cells:
        m = check_re_multi(cell, _re_all_flag.re)
        if m: return m.groups()[0]

In [ ]:
nb = read_nb("04_test.ipynb")
assert check_all_flag(nb['cells']) is None

In [ ]:
#hide
tst_flags_bck=Config().get('tst_flags')
try:
    Config()['tst_flags'] = 'fastai2|vslow'
    del _re_all_flag._re
    cells = [{'cell_type': cell_type, 'source': source} for cell_type, source in [
        ('code', '# export\nfrom local.core import *'), 
        ('markdown', '# title of some kind'), 
        ('code', '# all_vslow \n# all_fastai2'),
        ('code', '%nbdev_vslow_test all\n# all_fastai2'), # comment flag gets picked up before magic
        ('code', '%nbdev_vslow_test all '),
        ('code', '# all_fastai2'),
        ('code', '%nbdev_fastai2_test  all\n')]]
    for expected in ['vslow', 'fastai2', 'vslow', 'fastai2', 'fastai2']:
        test_eq(expected, check_all_flag(cells))
        cells.pop(2)
finally:
    Config()['tst_flags'] = tst_flags_bck
    del _re_all_flag._re

In [ ]:
#export
_re_flags = _ReTstFlags(False)

In [ ]:
# export
def get_cell_flags(cell):
    "Check for any special test flag in `cell`"
    if cell['cell_type'] != 'code' or len(Config().get('tst_flags',''))==0: return []
    return _re_flags.re[0].findall(cell['source']) + _re_flags.re[1].findall(cell['source'])

In [ ]:
test_eq(get_cell_flags({'cell_type': 'code', 'source': "#hide\n# fastai2\n"}), ['fastai2'])
test_eq(get_cell_flags({'cell_type': 'code', 'source': "#hide\n"}), [])

In [ ]:
#hide
for expected, flag in [(['fastai2'], 'fastai2'), ([], 'vslow')]:
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# {flag}\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n%nbdev_{flag}_test\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"# {flag}\n#hide\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_{flag}_test\n#hide\n")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell("#hide\n")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell(f"# all_{flag}")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_{flag}_test all")))
tst_flags_bck=Config().get('tst_flags')
try:
    Config()['tst_flags'] = 'fastai2|vslow'
    del _re_flags._re
    test_eq(['vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# vslow\n")))
    test_eq(['vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n%nbdev_vslow_test\n")))
    test_eq(['vslow', 'fastai2'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# vslow\n# fastai2")))
    test_eq(['fastai2', 'vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_fastai2_test\n%nbdev_vslow_test")))
finally:
    Config()['tst_flags'] = tst_flags_bck
    del _re_flags._re

## Testing a notebook

In [ ]:
# export
class NoExportPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that executes cells that are not exported and don't have a flag in `flags`"
    def __init__(self, flags, **kwargs):
        self.flags = flags
        super().__init__(**kwargs)

    def preprocess_cell(self, cell, resources, index):
        if 'source' not in cell or cell['cell_type'] != "code": return cell, resources
        for f in get_cell_flags(cell):
            if f not in self.flags:  return cell, resources
        res = super().preprocess_cell(cell, resources, index)
        return res

In [ ]:
# export
def test_nb(fn, flags=None):
    "Execute tests in notebook in `fn` with `flags`"
    os.environ["IN_TEST"] = '1'
    if flags is None: flags = []
    try:
        nb = read_nb(fn)
        all_flag = check_all_flag(nb['cells'])
        if all_flag is not None and all_flag not in flags: return
        mod = find_default_export(nb['cells'])
        ep = NoExportPreprocessor(flags, timeout=600, kernel_name='python3')
        pnb = nbformat.from_dict(nb)
        ep.preprocess(pnb)
    finally: os.environ.pop("IN_TEST")

## Export-

In [ ]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flags.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
